### Notebook demonstrating how to extract ts features using tsfresh
The number of features and data size in our dataset was too large for tsfresh to handle (memory constraint) so generally
our approach is to only look at a set of features (stride size in the prepare_batch method) at a time
The time to generate the tsfresh data is very large for our datasets so I've only tried it wiht 100,000 datapoints 

In [21]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [30]:
import xarray as xr
from tsfresh import extract_relevant_features, extract_features, feature_extraction
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import datetime

In [41]:
def clean_double_space(X_to_clean):
    print('On clean_double_space')
    rename_dict = {}
    for v in X_to_clean.variable.values:
        #if '__' in v:
        #    print('have ds:  ' + v)
        rename_dict[v] = v.replace('__', '_')
    X_to_clean = X_to_clean.assign_coords({'variable':list(rename_dict.values()) })
    return X_to_clean

In [3]:
def xr_batch_to_tsfresh_df(xr_batch):
    print('On xr_batch_to_tsfresh_df')
    #xr_batch = xr_batch.to_array()
    #xr_batch = xr_batch.dropna(dim='variable', how='all')
    stacked = xr_batch.stack(tsdata=('sample', 'time', 'variable')).to_dataframe()[['vars']]
    stacked.columns = ['value']
    stacked.reset_index(inplace=True)
    stacked.columns = ['id', 'time', 'kind', 'value']
    stacked.dropna(how='any', inplace=True)
    return stacked

In [4]:
def prepare_batch(X, offset=0, compute_tsfresh=False, known_relevant_features=None, n_jobs=62):    
    X = clean_double_space(X)
    print('On interpolate_na')
    X = X.chunk({'time': -1}).interpolate_na('time')
    X_tsfresh = xr_batch_to_tsfresh_df(X)
    assert(np.isnan(X_tsfresh['value']).any()==False)
    X_tsfresh = X_tsfresh[['id', 'time', 'kind', 'value']]
    num_ids = X_tsfresh['id'].max() + 1
    X_tsfresh['id'] = X_tsfresh['id'] + offset * num_ids
    
    #y['id'] = range(offset * num_ids, num_ids)
    
    feature_parameters = None
    if known_relevant_features is not None:       
        feature_parameters = feature_extraction.settings.from_columns(known_relevant_features)
    
    if not compute_tsfresh:
        return X_tsfresh
    
    relevant_features =[]
    stride = 75 
    #break this in to batches of variables to help is scale better in memory
    unique_kind = X_tsfresh['kind'].unique()
            
    for i in range(0,len(unique_kind),stride):
        print('On i: ' + str(i) + ' of ' + str(len(unique_kind)))
        #not every variable has a sample data so map labels to the data which does exist for that variable
        subset = X_tsfresh[X_tsfresh['kind'].isin(unique_kind[i:i+stride])]
        #return subset, unique_kind
        if feature_parameters is None:
            #filter y: y[y['id'].isin(subset.id.unique())][y_column]
            relevant_features.append( extract_features(subset, column_id='id', column_sort="time", column_kind="kind", column_value="value", n_jobs=n_jobs))
        else:
            relevant_features.append( extract_features(subset, kind_to_fc_parameters=feature_parameters, column_id='id', column_sort="time", column_kind="kind", column_value="value", n_jobs=n_jobs))
            
    
    relevant_features_df = pd.concat(relevant_features, axis=1)
    return relevant_features_df

In [5]:
X_train = xr.open_zarr('../Data/X_train_1.0.zarr/')

In [6]:
X_train

<xarray.Dataset>
Dimensions:    (sample: 252, time: 180, variable: 1131)
Coordinates:
    latitude   (sample) float64 dask.array<chunksize=(14,), meta=np.ndarray>
    longitude  (sample) float64 dask.array<chunksize=(39,), meta=np.ndarray>
  * variable   (variable) <U52 'ABSV_1000mb_avg' ... 'WILT_surface_min'
Dimensions without coordinates: sample, time
Data variables:
    vars       (sample, variable, time) float64 dask.array<chunksize=(32, 142, 45), meta=np.ndarray>

In [9]:
%time X_train_tf = prepare_batch(X_train, offset=0, compute_tsfresh=True)

On clean_double_space
have ds:  CWAT_entireatmosphere_consideredasasinglelayer__avg
have ds:  CWAT_entireatmosphere_consideredasasinglelayer__max
have ds:  CWAT_entireatmosphere_consideredasasinglelayer__min
have ds:  PWAT_entireatmosphere_consideredasasinglelayer__avg
have ds:  PWAT_entireatmosphere_consideredasasinglelayer__max
have ds:  PWAT_entireatmosphere_consideredasasinglelayer__min
have ds:  RH_entireatmosphere_consideredasasinglelayer__avg
have ds:  RH_entireatmosphere_consideredasasinglelayer__max
have ds:  RH_entireatmosphere_consideredasasinglelayer__min
have ds:  TOZNE_entireatmosphere_consideredasasinglelayer__avg
have ds:  TOZNE_entireatmosphere_consideredasasinglelayer__max
have ds:  TOZNE_entireatmosphere_consideredasasinglelayer__min
On interpolate_na
On xr_batch_to_tsfresh_df
On i: 0 of 1128


Feature Extraction: 100%|██████████| 309/309 [03:43<00:00,  1.38it/s]


On i: 75 of 1128


Feature Extraction: 100%|██████████| 309/309 [03:50<00:00,  1.34it/s]


On i: 150 of 1128


...

On i: 1050 of 1128


Feature Extraction: 100%|██████████| 310/310 [01:52<00:00,  2.77it/s]


On i: 1125 of 1128


Feature Extraction: 100%|██████████| 3/3 [00:00<00:00, 25.84it/s]


CPU times: user 10min 57s, sys: 3min 21s, total: 14min 19s
Wall time: 1h 3min 46s


In [10]:
def generate_prefix(val):
    tmp = val.split('_')
    return tmp[0] + '_' + tmp[1]

In [13]:
temp_path = '../Results/'

In [40]:
#offset = 0
def write_partitioned(prefix):
    #print('calculating cols: ' + prefix)
    cols_to_use = []
    for c in cols:
        if c.startswith(prefix):
            cols_to_use.append(c)
        
            
    #print('writing')
    X_train_tf[cols_to_use].to_parquet(temp_path + 'X_train_' + prefix + '_' + str(offset) + '.parquet', engine='pyarrow')

In [38]:
cols = X_train_tf.columns
prefix = pd.Series(cols.values).apply(generate_prefix).unique()

In [26]:
X_train_tf.head()

variable,ABSV_1000mb_avg__abs_energy,ABSV_1000mb_avg__absolute_sum_of_changes,"ABSV_1000mb_avg__agg_autocorrelation__f_agg_""mean""__maxlag_40","ABSV_1000mb_avg__agg_autocorrelation__f_agg_""median""__maxlag_40","ABSV_1000mb_avg__agg_autocorrelation__f_agg_""var""__maxlag_40","ABSV_1000mb_avg__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","ABSV_1000mb_avg__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","ABSV_1000mb_avg__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","ABSV_1000mb_avg__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","ABSV_1000mb_avg__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""",...,ICSEV_950mb_min__symmetry_looking__r_0.9500000000000001,ICSEV_950mb_min__time_reversal_asymmetry_statistic__lag_1,ICSEV_950mb_min__time_reversal_asymmetry_statistic__lag_2,ICSEV_950mb_min__time_reversal_asymmetry_statistic__lag_3,ICSEV_950mb_min__value_count__value_-1,ICSEV_950mb_min__value_count__value_0,ICSEV_950mb_min__value_count__value_1,ICSEV_950mb_min__variance,ICSEV_950mb_min__variance_larger_than_standard_deviation,ICSEV_950mb_min__variation_coefficient
id,,,,,,,,,,,,,,,,,,,,,
0,1.922320e-07,0.000314,0.005289,-0.023179,0.105949,0.000094,0.000075,0.000051,1.593620e-10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.098714e-07,0.000508,-0.086704,-0.128088,0.076983,0.000119,0.000110,0.000100,6.753296e-12,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.665041e-07,0.000547,-0.041518,-0.030160,0.031853,0.000121,0.000103,0.000092,7.747075e-11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.942668e-07,0.000850,0.041993,0.009941,0.266976,0.000136,0.000099,0.000052,5.976774e-10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.413081e-07,0.001002,-0.004682,-0.011084,0.049631,0.000118,0.000091,0.000057,2.496842e-10,0.000156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
data = Parallel(n_jobs=1)(map(delayed(write_partitioned), prefix))

calculating cols: ABSV_1000mb
writing
calculating cols: ABSV_100mb
writing
calculating cols: ABSV_10mb
writing
calculating cols: ABSV_150mb
writing
calculating cols: ABSV_200mb
writing
calculating cols: ABSV_20mb
writing
calculating cols: ABSV_250mb
writing
calculating cols: ABSV_300mb
writing
calculating cols: ABSV_30mb
writing
calculating cols: ABSV_350mb
writing
calculating cols: ABSV_400mb
writing
calculating cols: ABSV_450mb
writing
calculating cols: ABSV_500mb
writing
calculating cols: ABSV_50mb
writing
calculating cols: ABSV_550mb
writing
calculating cols: ABSV_600mb
writing
calculating cols: ABSV_650mb
writing
calculating cols: ABSV_700mb
writing
calculating cols: ABSV_70mb
writing
calculating cols: ABSV_750mb
writing
calculating cols: ABSV_800mb
writing
calculating cols: ABSV_850mb
writing
calculating cols: ABSV_900mb
writing
calculating cols: ABSV_925mb
writing
calculating cols: ABSV_950mb
writing
calculating cols: ABSV_975mb
writing
calculating cols: APTMP_2maboveground
writ

calculating cols: TOZNE_entireatmosphere
writing
calculating cols: TSOIL_0D1M0D4mbelowground
writing
calculating cols: TSOIL_0D4M1mbelowground
writing
calculating cols: TSOIL_0M0D1mbelowground
writing
calculating cols: TSOIL_1M2mbelowground
writing
calculating cols: UGRD_0D995sigmalevel
writing
calculating cols: UGRD_1000mb
writing
calculating cols: UGRD_100maboveground
writing
calculating cols: UGRD_100mb
writing
calculating cols: UGRD_10maboveground
writing
calculating cols: UGRD_10mb
writing
calculating cols: UGRD_150mb
writing
calculating cols: UGRD_1829mabovemeansealevel
writing
calculating cols: UGRD_200mb
writing
calculating cols: UGRD_20mb
writing
calculating cols: UGRD_250mb
writing
calculating cols: UGRD_2743mabovemeansealevel
writing
calculating cols: UGRD_300mb
writing
calculating cols: UGRD_30M0mbaboveground
writing
calculating cols: UGRD_30mb
writing
calculating cols: UGRD_350mb
writing
calculating cols: UGRD_3658mabovemeansealevel
writing
calculating cols: UGRD_400mb
wri

In [28]:
pd.Series(prefix).to_csv(temp_path + '/prefix2.csv', index=False)

In [36]:
cols

Index(['ABSV_1000mb_avg__abs_energy',
       'ABSV_1000mb_avg__absolute_sum_of_changes',
       'ABSV_1000mb_avg__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'ABSV_1000mb_avg__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'ABSV_1000mb_avg__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'ABSV_1000mb_avg__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"max"',
       'ABSV_1000mb_avg__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"',
       'ABSV_1000mb_avg__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',
       'ABSV_1000mb_avg__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',
       'ABSV_1000mb_avg__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',
       ...
       'ICSEV_950mb_min__symmetry_looking__r_0.9500000000000001',
       'ICSEV_950mb_min__time_reversal_asymmetry_statistic__lag_1',
       'ICSEV_950mb_min__time_reversal_asymmetry_statistic__lag_2',
       'ICSEV_950mb_min__time_rever

In [35]:
X_train_tf.columns

Index(['ABSV_1000mb_avg__abs_energy',
       'ABSV_1000mb_avg__absolute_sum_of_changes',
       'ABSV_1000mb_avg__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'ABSV_1000mb_avg__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'ABSV_1000mb_avg__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'ABSV_1000mb_avg__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"max"',
       'ABSV_1000mb_avg__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"',
       'ABSV_1000mb_avg__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',
       'ABSV_1000mb_avg__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',
       'ABSV_1000mb_avg__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',
       ...
       'VWSH_PV_EQ_M2eM06_Km_2_kg_s_surface_min__symmetry_looking__r_0.9500000000000001',
       'VWSH_PV_EQ_M2eM06_Km_2_kg_s_surface_min__time_reversal_asymmetry_statistic__lag_1',
       'VWSH_PV_EQ_M2eM06_Km_2_kg_s_surface_min__time_r

In [42]:
for i in range(3,40):
    X_train = xr.open_zarr('../Data/X_train_' + str(i) + '.0.zarr/')
    print(str(datetime.datetime.now()) + ' On ' + str(i))
    X_train_tf = prepare_batch(X_train, offset=i, compute_tsfresh=True)
    cols = X_train_tf.columns
    prefix = pd.Series(cols.values).apply(generate_prefix).unique()
    offset = i
    Parallel(n_jobs=1)(map(delayed(write_partitioned), prefix))
    pd.Series(prefix).to_csv(temp_path + '/prefix_' + str(offset) + '.csv', index=False)

2020-06-13 20:30:35.691212 On 3
On clean_double_space
On interpolate_na
On xr_batch_to_tsfresh_df
On i: 0 of 1125


Feature Extraction: 100%|██████████| 310/310 [04:06<00:00,  1.26it/s]


On i: 75 of 1125


Feature Extraction: 100%|██████████| 310/310 [03:53<00:00,  1.33it/s]


On i: 150 of 1125


...

Feature Extraction: 100%|██████████| 310/310 [04:05<00:00,  1.26it/s]


On i: 1050 of 1128


Feature Extraction: 100%|██████████| 306/306 [01:36<00:00,  3.19it/s]


On i: 1125 of 1128


Feature Extraction: 100%|██████████| 3/3 [00:00<00:00, 18.73it/s]


In [2]:
X = pd.read_parquet('/home/scottcha/X_train.parquet')

In [3]:
X.head()

,id,time,kind,value
88218,0,78,ABSV_1000mb_avg,0.000030
88219,0,78,ABSV_1000mb_max,0.000070
88220,0,78,ABSV_1000mb_min,-0.000038
88221,0,78,ABSV_100mb_avg,0.000098
88222,0,78,ABSV_100mb_max,0.000158


In [4]:
y = pd.read_parquet('/home/scottcha/y_train.parquet')

In [10]:
len(X.id.unique())

2026

In [11]:
y = y.iloc[X.id.unique()]

In [ ]:
relevant_features = extract_relevant_features(X, y['Day1DangerAboveTreeline'], column_id='id', column_sort="time", column_kind="kind", column_value="value", n_jobs=60)

Feature Extraction:  22%|██▏       | 66/300 [2:35:27<21:18:50, 327.91s/it] 

In [13]:
relevant_features.to_parquet('/home/scottcha/X_train_relevant.parquet')

In [14]:
X_t = pd.read_parquet('/home/scottcha/X_test.parquet')

In [17]:
X_t.tail()

,id,time,kind,value
45066229,255,179,HGT_PV_EQ_2eM06_Km_2_kg_s_surface_avg,12104.357609
45066230,255,179,PRES_PV_EQ_2eM06_Km_2_kg_s_surface_avg,19250.961815
45066231,255,179,VWSH_PV_EQ_2eM06_Km_2_kg_s_surface_avg,-0.000287
45066238,255,179,PRMSL_meansealevel_avg,101093.568530
45066239,255,179,N5WAVH_500mb_avg,5670.672867


In [18]:
feature_parameters = feature_extraction.settings.from_columns(relevant_features)

In [19]:
relevant_features_test = extract_features(X_t, kind_to_fc_parameters=feature_parameters, column_id='id', column_sort="time", column_kind="kind", column_value="value", n_jobs=60)

Feature Extraction: 100%|██████████| 300/300 [1:57:39<00:00, 23.53s/it]  


In [21]:
relevant_features_test.to_parquet('/home/scottcha/X_test.relevant.parquet')